### import the relevant libraries

In [2]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

### load the data 

In [3]:
mnist, mnist_info = tfds.load( name="mnist", as_supervised= True,with_info=True )

Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to ~/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='~/tensorflow_datasets/mnist/3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [5]:
train ,test = mnist["train"], mnist["test"]

In [36]:
valid_num = 0.1 *mnist_info.splits["train"].num_examples
valid_num = tf.cast(valid_num, tf.int64)

In [158]:
# func for scaling
def scale(image,label):
  image =tf.cast(image,tf.float64)
  image/=255.
  return image , label
#scaling the data
train_scaled=train.map(scale)
test_scaled= test.map(scale)


In [160]:
train_scaled

<MapDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [161]:
#shauffeling
BUFFER_SIZE = 1000
BATCH_SIZE = 100
train_shuffeled = train_scaled.shuffle( BUFFER_SIZE)

In [162]:
valid_data = train_shuffeled.take(valid_num)

In [163]:
list(valid_data.as_numpy_iterator())[0][1]# to see the arrays

3

In [164]:
# how to rewtite train data to be without the validation
train_final = train_shuffeled.skip(valid_num) # skip the first x number

In [165]:
train_final = train_final.batch(BATCH_SIZE)
valid_data = valid_data.batch(valid_num)

In [166]:
list(train_final.as_numpy_iterator())[0][1]

array([3, 8, 6, 9, 4, 1, 3, 7, 4, 4, 9, 0, 6, 1, 9, 1, 9, 6, 8, 1, 7, 0,
       2, 5, 7, 0, 5, 6, 1, 1, 5, 3, 4, 3, 1, 1, 2, 0, 4, 4, 0, 2, 4, 9,
       3, 0, 9, 7, 4, 8, 9, 5, 5, 1, 3, 2, 0, 1, 0, 2, 7, 1, 7, 9, 7, 3,
       5, 9, 9, 1, 2, 3, 1, 0, 2, 0, 2, 0, 8, 1, 5, 7, 8, 6, 3, 9, 7, 0,
       6, 7, 5, 3, 8, 0, 9, 4, 2, 6, 2, 7])

In [167]:
valid_x ,valid_y = next(iter(valid_data))

### create a model

In [152]:
input_size = 28**2
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1), dtype =tf.float64),
    tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax")

])


In [153]:
model.compile(
    loss= "sparse_categorical_crossentropy",
    optimizer = "adam",
    metrics = "accuracy"
)

In [148]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 784)               0         
                                                                 
 dense_20 (Dense)            (None, 50)                39250     
                                                                 
 dense_21 (Dense)            (None, 50)                2550      
                                                                 
 dense_22 (Dense)            (None, 10)                510       
                                                                 
Total params: 42,310
Trainable params: 42,310
Non-trainable params: 0
_________________________________________________________________


In [168]:
history = model.fit(train_final,
          epochs = 5, verbose=2
          )

Epoch 1/5
540/540 - 5s - loss: 0.0842 - accuracy: 0.9745 - 5s/epoch - 9ms/step
Epoch 2/5
540/540 - 5s - loss: 0.0719 - accuracy: 0.9785 - 5s/epoch - 9ms/step
Epoch 3/5
540/540 - 5s - loss: 0.0628 - accuracy: 0.9813 - 5s/epoch - 8ms/step
Epoch 4/5
540/540 - 5s - loss: 0.0575 - accuracy: 0.9820 - 5s/epoch - 9ms/step
Epoch 5/5
540/540 - 5s - loss: 0.0502 - accuracy: 0.9845 - 5s/epoch - 9ms/step


In [157]:
train_data, train

(<BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>)